In [11]:
import astropy
from astropy import units
from astropy.coordinates import SkyCoord
from astropy.table import Table
import scipy
import numpy as np
from tqdm import tqdm

yr_to_s = 365*24*3600

def GW_A(m1_msun,m2_msun,P,d_pc):
    m1 = m1_msun*astropy.constants.M_sun.value
    m2 = m2_msun*astropy.constants.M_sun.value

    d = d_pc*astropy.constants.pc.value
    
    f = 2/P
    
    M = (m1*m2)**(3./5)/(m1+m2)**(1./5)
    A = 2*(astropy.constants.G.value*M)**(5./3)/astropy.constants.c.value**4/d*(np.pi*f)**(2./3)
    Tobs = 4*yr_to_s
    Ncycle = f*Tobs
    strain = np.sqrt(Ncycle)*A
    return strain

lisa_sens = np.loadtxt('01_Data/lisa_sens.csv')
lisa_sens_interp = scipy.interpolate.interp1d(lisa_sens[:,0],lisa_sens[:,1])


In [12]:

lisa_source = 0

for n_seed in tqdm(range(10)):
    dataset = Table.read('./02_results/GalacticDWDBeaming.ecsv')
    dataset = dataset[(-0.03<dataset['beaming_falseAlarmProbab'][:,n_seed])*\
                      (dataset['beaming_falseAlarmProbab'][:,n_seed]<1e-5)*\
                      (dataset['r']<20)]
    
    recoverability = []
    recoverability_index = []
    
    for ii_dwd in tqdm(range(len(dataset))):
        dwd = dataset[ii_dwd]
        Period = dwd['period_hour']*units.hour
        M1 = dwd['M1']*units.M_sun
        M2 = dwd['M2']*units.M_sun
        R1 = dwd['R1']*units.R_sun
        R2 = dwd['R2']*units.R_sun
        T1 = 10**dwd['logTeff1']*units.K
        T2 = 10**dwd['logTeff2']*units.K
        inclination = dwd['inclination']*units.degree
        source_magnitude = {'u':dwd['u'],'g':dwd['g'],'r':dwd['r'],'i':dwd['i'],'z':dwd['z'],'y':dwd['y']}
    
        source_RA = dwd['RA_ICRS']*units.degree
        source_DEC = dwd['DEC_ICRS']*units.degree
        source_dkpc = dwd['d_ICRS']*units.kiloparsec
    
        source_coordinate = SkyCoord(source_RA,source_DEC,frame='icrs')
    
        observation_filters = list('ugrizy')
    
        params = [Period,source_dkpc]
        
        strain = GW_A(M1.value,M2.value,Period.value*3600,\
                      source_dkpc.value*1000)
        
        lisa_sensCurve = lisa_sens_interp(2/(Period.value*3600))
        
        if strain > lisa_sensCurve:
            lisa_source += 1
            recoverability.append(params)

lisa_source = lisa_source/10

print(lisa_source)

100%|███████████████████████████████████████████| 10/10 [00:25<00:00,  2.53s/it]

11.8


In [13]:
lisa_source

11.8

In [4]:
recoverability

[[<Quantity 0.54764724 h>, <Quantity 0.62939453 kpc>],
 [<Quantity 1.5669093 h>, <Quantity 0.31567383 kpc>],
 [<Quantity 0.69228834 h>, <Quantity 0.5605469 kpc>],
 [<Quantity 1.6848288 h>, <Quantity 0.13378906 kpc>],
 [<Quantity 1.5367815 h>, <Quantity 0.20593262 kpc>],
 [<Quantity 2.1249323 h>, <Quantity 0.15454102 kpc>],
 [<Quantity 1.6122314 h>, <Quantity 0.14501953 kpc>]]

In [14]:
11.8*4

47.2